# AMPL Capacitated p-Median Problem with GCG
[![cpmp.ipynb](https://img.shields.io/badge/github-%23121011.svg?logo=github)](https://github.com/ampl/amplcolab/blob/master/lentz/gcg/cpmp.ipynb) [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://github.com/ampl/amplcolab/blob/master/lentz/gcg/cpmp.ipynb) [![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/ampl/amplcolab/blob/master/lentz/gcg/cpmp.ipynb) [![Gradient](https://assets.paperspace.io/img/gradient-badge.svg)](https://console.paperspace.com/github/ampl/amplcolab/blob/master/lentz/gcg/cpmp.ipynb) [![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/ampl/amplcolab/blob/master/lentz/gcg/cpmp.ipynb) [![Hits](https://h.ampl.com/https://github.com/ampl/amplcolab/blob/master/lentz/gcg/cpmp.ipynb)](https://colab.ampl.com)

Description: Dantzig-Wolfe decomposition for Capacitated p-Median Problem with GCG

Tags: GCG, cpmp, amplpy, dantzig-wolfe decomposition, branch-price-and-cut, highlights

Notebook author: Jurgen Lentz <<jurgenlentz26@gmail.com>>

In [1]:
# Install dependencies
!pip install -q amplpy

In [2]:
# Google Colab & Kaggle integration
from amplpy import AMPL, tools
ampl = tools.ampl_notebook(
    modules=["gurobi", "scip", "gcg"], # modules to install
    license_uuid="04da0909-bbeb-4a38-98d2-ba75e4850f5c") # license to use

Licensed to Development License for Jurgen Lentz <jurgenlentz26@gmail.com>.


## Capacitated $p$-median problem

In [3]:
%%ampl_eval
param n;
param C;

suffix master IN, binary;
suffix block IN, integer;

set I = 1..n ordered;
param w {I} > 0;
param maxVal := max {i in I} w[i];
param maxbins := ceil(n / floor(C / maxVal));

set J = 1..maxbins;

var x {I,J} binary;
var y {J} binary;

minimize Cost:  sum {j in J} y[j];

subject to b_Capacity {j in J}:
   sum {i in I} w[i] * x[i,j] <= C * y[j] suffix block j;

subject to m_Allocate {i in I}:
   sum {j in J} x[i,j] >= 1 suffix master 1;


We generate a small instance with 5 locations and 2 locations as medians.

In [4]:
ampl.param["n"] = 180
ampl.param["C"] = 1000

w = [1, 514, 515, 4, 523, 12, 526, 15, 16, 531, 20, 20, 21, 24, 24, 516, 26, 35, 36, 36, 550, 550, 555, 44, 557, 558, 51, 54, 58, 572, 61, 61, 574, 575, 67, 68, 69, 582, 585, 586, 587, 588, 78, 592, 83, 84, 598, 91, 698, 95, 97, 97, 610, 614, 102, 102, 105, 106, 109, 622, 622, 112, 117, 596, 617, 634, 639, 131, 645, 645, 647, 137, 139, 656, 147, 660, 152, 665, 667, 157, 670, 671, 671, 160, 162, 675, 164, 169, 170, 685, 686, 178, 694, 695, 696, 186, 189, 193, 195, 205, 209, 215, 216, 216, 218, 229, 240, 240, 243, 690, 251, 252, 256, 270, 274, 279, 283, 283, 291, 293, 294, 298, 305, 312, 320, 322, 325, 325, 327, 333, 339, 346, 351, 358, 359, 361, 367, 372, 373, 374, 377, 379, 385, 388, 389, 391, 399, 409, 413, 415, 416, 423, 433, 435, 435, 440, 446, 447, 672, 453, 465, 466, 466, 467, 468, 472, 475, 481, 485, 492, 495, 495, 496, 497, 497, 498, 681, 681, 505, 508]

ampl.param["w"] = {
  i: w[i-1]
  for i in range(1,len(w)+1)
}

ampl.param["n"] = len(w)

## Automatic Mode in GCG with AMPL

We use AMPL to call the solver GCG to solve our CPMP instance automatically without providing any information about the Dantzig-Wolfe decomposition. Here, GCG detects different decompositions and chooses heuristically the best decomposition. Afterwards, the solver uses a branch-price-and-cut algorithm to solve it to optimality.

In [5]:
ampl.option["solver"] = "gurobi"
ampl.option["gurobi_options"] = "outlev=1"
ampl.solve()

Gurobi 10.0.1: Set parameter LogToConsole to value 1
tech:outlev=1
Set parameter InfUnbdInfo to value 1
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-5350U CPU @ 1.80GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 360 rows, 32580 columns and 64980 nonzeros
Model fingerprint: 0xc7d9e6d8
Variable types: 0 continuous, 32580 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 130.0000000
Presolve time: 0.31s
Presolved: 360 rows, 32580 columns, 64980 nonzeros
Variable types: 0 continuous, 32580 integer (32580 binary)

Root relaxation: objective 6.399300e+01, 1340 iterations, 0.18 seconds (0.12 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    Be

In [5]:
ampl.option["solver"] = "gcg"
ampl.option["gcg_options"] = "outlev=1"
ampl.solve()

GCG 3.6.0: tech:outlev=1
 added complete decomp for original problem with 180 blocks and 180 masterconss, 0 linkingvars, 0 mastervars, and max white score of   0.497222 
presolving:
(round 1, exhaustive) 0 del vars, 0 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 180 upgd conss, 0 impls, 3981 clqs
(round 2, exhaustive) 0 del vars, 0 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 360 upgd conss, 0 impls, 3981 clqs
   (13.6s) probing: 1000/32580 (3.1%) - 0 fixings, 0 aggregations, 19850 implications, 0 bound changes
   (13.6s) probing: 1001/32580 (3.1%) - 0 fixings, 0 aggregations, 19851 implications, 0 bound changes
   (13.6s) probing aborted: 1000/1000 successive useless probings
presolving (3 rounds: 3 fast, 3 medium, 3 exhaustive):
 0 deleted vars, 0 deleted constraints, 0 added constraints, 0 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 3982 cliques
presolved problem has 32580 variables (32580 bin, 0 i

## Using a custom decomposition in GCG with AMPL

AMPL allows the users to create their own decomposition and forwards it to GCG using suffixes. Here, we assign the allocation and pmedian constraints to the master/linking constraints and each capacity constraint to a different pricing problem.

In [ ]:
ampl.solve()

Analogously, the user can fix constraints as pricing block or master constraint and variables as pricing block, master or linking variables. It is not needed to provide a complete decomposition. If the user provides a partial decomposition, GCG completes the decomposition by fixing only the left constraints and variables using its detection loop.

**_NOTE:_**  The index of pricing block constraints/variables starts at 1.